In [4]:
import pandas as pd
from datetime import datetime, timedelta

In [56]:
def price_storage_contract(
    df_prices,
    injection_schedule,
    withdrawal_schedule,
    injection_cost_per_mmbtu,
    withdrawal_cost_per_mmbtu,
    storage_cost_per_month,
    max_storage_volume,
    max_injection_rate_per_day,
    max_withdrawal_rate_per_day
):
    # Normalize price data
    df_prices['Dates'] = pd.to_datetime(df_prices['Dates']).dt.date
    price_dict = df_prices.set_index('Dates')['Prices'].to_dict()

    total_value = 0.0
    current_storage = 0.0
    all_dates = set()

    # Process injections
    for entry in injection_schedule:
        date = pd.to_datetime(entry['date']).date()
        volume = entry['volume']
        all_dates.add(date)

        if volume > max_injection_rate_per_day:
            raise ValueError(f"Injection rate too high on {date}")
        if current_storage + volume > max_storage_volume:
            raise ValueError(f"Storage overflow on {date}")
        if date not in price_dict:
            raise ValueError(f"No price available for injection date {date}")

        price = price_dict[date]
        total_value -= volume * price  # cost to buy
        total_value -= injection_cost_per_mmbtu * volume
        current_storage += volume

    # Process withdrawals
    for entry in withdrawal_schedule:
        date = pd.to_datetime(entry['date']).date()
        volume = entry['volume']
        all_dates.add(date)

        if volume > max_withdrawal_rate_per_day:
            raise ValueError(f"Withdrawal rate too high on {date}")
        if current_storage - volume < 0:
            raise ValueError(f"Not enough gas to withdraw on {date}")
        if date not in price_dict:
            raise ValueError(f"No price available for withdrawal date {date}")

        price = price_dict[date]
        total_value += volume * price  # revenue from selling
        total_value -= withdrawal_cost_per_mmbtu * volume
        current_storage -= volume

    # Calculate storage duration and cost
    all_dates = sorted(all_dates)
    if all_dates:
        storage_months = max(1, ((all_dates[-1] - all_dates[0]).days + 1) // 30)
        total_value -= storage_months * storage_cost_per_month

    return total_value


In [58]:
#sample data
#df = pd.read_csv("Nat_Gas.csv")
data = {  'Dates': ['2024-06-01', '2024-10-01'],    'Prices': [2.0, 3.5] }
df = pd.DataFrame(data)
injection_schedule = [{'date': '2024-06-01', 'volume': 1e6}]
withdrawal_schedule = [{'date': '2024-10-01', 'volume': 1e6}]


value = price_storage_contract(
    df,
    injection_schedule,
    withdrawal_schedule,
    injection_cost_per_mmbtu=0.01,
    withdrawal_cost_per_mmbtu=0.01,
    storage_cost_per_month=100_000,
    max_storage_volume=1e6,
    max_injection_rate_per_day=1e6,
    max_withdrawal_rate_per_day=1e6
)

print(f"Contract Value: ${value:,}")

Contract Value: $1,080,000.0
